### Create a recomendation engine based on my spotify library. Then recommend songs from a playlist of 10,000 songs 

-------------------------

### Useful links

#### Retrieving spotify data from spotify dev api
https://towardsdatascience.com/how-to-create-large-music-datasets-using-spotipy-40e7242cc6a6

https://github.com/MaxHilsdorf/introduction_to_spotipy

#### Retrieving your own personal spotify data
https://towardsdatascience.com/get-your-spotify-streaming-history-with-python-d5a208bbcbd3

#### Spotify API 
https://spotipy.readthedocs.io/en/latest/#more-examples

https://developer.spotify.com/documentation/web-api/

https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/



In [1]:
import pandas as pd
import numpy as np

In [3]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [128]:
import spotipy
import spotipy.util as util

In [129]:
CLIENT_ID = "1c2efcd1b3014dfbb795b1ed9833e6fb"
CLIENT_SECRET = "998d7326d90a43048df85781eba855cd"

In [130]:
token = spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
cache_token = token.get_access_token()
sp = spotipy.Spotify(cache_token)

C:\Users\charl\anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  


In [131]:
playlist_creator = "spotify"
playlist = '4rnleEAOdmFAbRcNCgZMpY'

In [132]:
test = sp.user_playlist_tracks("spotify", biggest_playist_ever)

NameError: name 'biggest_playist_ever' is not defined

In [43]:
def analyze_playlist(creator, playlist_id):
    
    # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = sp.user_playlist_tracks(creator, playlist_id)["items"]
    for track in playlist:
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [ ]:
df = analyze_playlist(playlist_creator,playlist)

In [42]:
len(df)

100

In [45]:
#https://stackoverflow.com/questions/39086287/spotipy-how-to-read-more-than-100-tracks-from-a-playlist
def get_playlist_tracks(username,playlist_id):
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

In [46]:
test1 = get_playlist_tracks(playlist_creator,playlist)

In [53]:
def combined(username,playlist_id):
     # Create empty dataframe
    playlist_features_list = ["artist", "album", "track_name", "track_id", 
                             "danceability", "energy", "key", "loudness", "mode", "speechiness",
                             "instrumentalness", "liveness", "valence", "tempo", "duration_ms", "time_signature"]
    playlist_df = pd.DataFrame(columns = playlist_features_list)
    
    # Create empty dict
    playlist_features = {}
    
    # Loop through every track in the playlist, extract features and append the features to the playlist df
    playlist = get_playlist_tracks(username,playlist_id)
    for track in playlist:
        # Get metadata
        playlist_features["artist"] = track["track"]["album"]["artists"][0]["name"]
        playlist_features["album"] = track["track"]["album"]["name"]
        playlist_features["track_name"] = track["track"]["name"]
        playlist_features["track_id"] = track["track"]["id"]
        # Get audio features
        audio_features = sp.audio_features(playlist_features["track_id"])[0]
        for feature in playlist_features_list[4:]:
            playlist_features[feature] = audio_features[feature]
        
        # Concat the dfs
        track_df = pd.DataFrame(playlist_features, index = [0])
        playlist_df = pd.concat([playlist_df, track_df], ignore_index = True)
        
    return playlist_df

In [55]:
a = combined(playlist_creator,playlist)

In [56]:
len(a)

4778

In [57]:
a.head()

,artist,album,track_name,track_id,danceability,energy,key,loudness,mode,speechiness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Hozier,Hozier (Deluxe),Take Me To Church,7dS5EaCoMnN7DzlpT6aRn2,0.566,0.664,4,-5.303,0,0.0464,0,0.116,0.437,128.945,241688,4
1,Mike Posner,31 Minutes to Takeoff,Cooler Than Me - Single Mix,2V4bv1fNWfTcyRJKmej6Sj,0.768,0.820,7,-4.630,0,0.0475,0,0.704,0.625,129.969,213293,4
2,"Tyler, The Creator",Flower Boy,See You Again (feat. Kali Uchis),7KA4W4McWYRpgf0fWsJZWB,0.558,0.559,6,-9.222,1,0.0959,7.49e-06,0.109,0.620,78.558,180387,4
3,Bastille,Bad Blood,Pompeii,3gbBpTdY8lnQwqxNCcf795,0.679,0.715,9,-6.383,1,0.0407,0,0.271,0.571,127.435,214148,4
4,Shakira,"Oral Fixation, Vol. 2 (Expanded Edition)",Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,0.778,0.824,10,-5.892,0,0.0712,0,0.405,0.756,100.024,218093,4


In [58]:
a.to_csv('biggest_playlist_ever.csv')

### Now time to bring in my personal song data

In [176]:
import json 
  
# Opening JSON file 
f = open(r'C:\Users\charl\Python\GitHub Projects\Spotify_Project\MyData\StreamingHistory0.json',encoding="utf8") 
  
# returns JSON object as  
# a dictionary 
data = json.load(f) 

In [134]:
hist = pd.DataFrame.from_dict(data)

In [135]:
hist.head()

,endTime,artistName,trackName,msPlayed
0,2019-09-20 18:05,Elvis Presley,From a Jack to a King,147973
1,2019-09-20 18:08,Elvis Presley,From a Jack to a King,147973
2,2019-09-20 18:26,Marty Robbins,Devil Woman,130966
3,2019-09-21 17:49,Marty Robbins,Devil Woman,6843
4,2019-09-21 17:49,Jimi Hendrix,May This Be Love,39967


In [136]:
hist.to_csv('streaming_history.csv')

### Get features for all songs that I listed to in 2019

In [137]:
my_songs = pd.read_csv(r'C:\Users\charl\Python\GitHub Projects\Spotify_Project\MyData\my_2019_tracks.csv')

In [139]:
my_songs

,Unnamed: 0,trackName,artistName,skip,play,like
0,0,"""C"" is for Cookie",Cookie Monster,1,0,0
1,1,$ave Dat Money (feat. Fetty Wap & Rich Homie Q...,Lil Dicky,1,1,0
2,2,'Round Here,IODONTPLAY,0,2,1
3,3,'Till I Collapse,Eminem,1,1,0
4,4,(Bonus) Air Canada,RiFF Raff & DOLLABiLLGATES,0,2,1
...,...,...,...,...,...,...
2625,2625,wonderful,Travis Scott,2,2,0
2626,2626,À toi,Léonie Deval,0,1,1
2627,2627,"Études, Op. 10: Etude No. 6 in E-Flat Minor, O...",Frédéric Chopin,0,1,1
2628,2628,• TEMPTED,Jazz Cartier,1,3,1


In [167]:
songs= my_songs['trackName']
artists= my_songs['artistName']

combined = list(zip(songs,artists))
combined[0]

('"C" is for Cookie', 'Cookie Monster')

In [209]:
def get_track_id(zipped):
    #Get track info
    artist = zipped[1]
    track = zipped[0]
    track_id = sp.search(q='artist:' + artist + ' track:' + track, type='track')
    
    #Sort through the dict
    track = track_id['tracks']['items']
    
    #The dict stops and turns into a shitty untraversable list so we need to be creative
    #Turn into string and pull all characters after 'tracks' appears in the url
    mystring =  str(track)
    keyword = '/tracks/'
    before_keyword, keyword, after_keyword = mystring.partition(keyword)
    
    #Extract the song id
    lsty = list(after_keyword.split(","))
    track_id = lsty[0][0:-1]

    track_id
    return track_id

In [210]:
song_ids=[]
for i in combined:
    track_id = get_track_id(i)
    song_ids.append(track_id)

In [212]:
sp.audio_features(tracks=song_ids[0])

[{'danceability': 0.604,
  'energy': 0.902,
  'key': 11,
  'loudness': -8.856,
  'mode': 1,
  'speechiness': 0.217,
  'acousticness': 0.511,
  'instrumentalness': 0,
  'liveness': 0.36,
  'valence': 0.701,
  'tempo': 119.23,
  'type': 'audio_features',
  'id': '6EVN97RIyZBCegsYyEnkMv',
  'uri': 'spotify:track:6EVN97RIyZBCegsYyEnkMv',
  'track_href': 'https://api.spotify.com/v1/tracks/6EVN97RIyZBCegsYyEnkMv',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6EVN97RIyZBCegsYyEnkMv',
  'duration_ms': 89205,
  'time_signature': 4}]

### Now create a datafram with these columns, append all the track info, then delete the uncessessary columns